In [1]:
%load_ext dotenv
%dotenv ../brainstation_capstone_cfg.env

In [2]:
import requests
import base64
import os

In [3]:
# Replace these values with your own
CLIENT_ID = os.environ["SPOTIPY_CLIENT_ID"]
CLIENT_SECRET =  os.environ["SPOTIPY_CLIENT_SECRET"]

In [ ]:
# Encode client credentials for authorization
client_creds = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

# API endpoints
AUTH_URL = "https://accounts.spotify.com/api/token"
PLAYLISTS_URL = "https://api.spotify.com/v1/me/playlists"

# Request access token
auth_response = requests.post(
    AUTH_URL,
    data={"grant_type": "client_credentials"},
    headers={"Authorization": f"Basic {client_creds_b64}"}
)
auth_data = auth_response.json()
access_token = auth_data['access_token']
# Create a private playlist
playlist_name = "My Private Playlist"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
data = {
    "name": playlist_name,
    "public": False  # Set to False to make it private
}
response = requests.post(
    PLAYLISTS_URL,
    json=data,
    headers=headers
)

if response.status_code == 201:
    print(f"Private playlist '{playlist_name}' created successfully.")
else:
    print("Playlist creation failed.")

In [ ]:
import requests
import base64

# Replace these values with your own
CLIENT_ID = "your_client_id"
CLIENT_SECRET = "your_client_secret"

# Encode client credentials for authorization
client_creds = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

# API endpoints
AUTH_URL = "https://accounts.spotify.com/api/token"
PLAYLISTS_URL = "https://api.spotify.com/v1/me/playlists"

# Request access token
auth_response = requests.post(
    AUTH_URL,
    data={"grant_type": "client_credentials"},
    headers={"Authorization": f"Basic {client_creds_b64}"}
)

auth_data = auth_response.json()
if "access_token" in auth_data:
    access_token = auth_data['access_token']
    print("Access token acquired successfully.")
else:
    print("Failed to acquire access token:", auth_data)

# Create a private playlist
playlist_name = "My Private Playlist"
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}
data = {
    "name": playlist_name,
    "public": False
}
response = requests.post(
    PLAYLISTS_URL,
    json=data,
    headers=headers
)

print("Response Status Code:", response.status_code)
print("Response JSON:", response.json())


In [ ]:
from flask import Flask, request, redirect
import requests
import base64

app = Flask(__name__)

# Replace these values with your own
CLIENT_ID = "your_client_id"
CLIENT_SECRET = "your_client_secret"
REDIRECT_URI = "http://localhost:5000/callback"  # Update with your redirect URI
SCOPE = "playlist-modify-private"

# Step 1: Redirect the user to the authorization URL
@app.route("/login")
def login():
    auth_url = (
        f"https://accounts.spotify.com/authorize"
        f"?client_id={CLIENT_ID}"
        f"&response_type=code"
        f"&redirect_uri={REDIRECT_URI}"
        f"&scope={SCOPE}"
    )
    return redirect(auth_url)

# Step 2: Handle the authorization code and exchange for an access token
@app.route("/callback")
def callback():
    auth_code = request.args.get("code")
    
    # Exchange authorization code for access token
    auth_headers = {
        "Authorization": f"Basic {base64.b64encode((CLIENT_ID + ':' + CLIENT_SECRET).encode()).decode()}"
    }
    auth_data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": REDIRECT_URI,
    }
    
    auth_response = requests.post(
        "https://accounts.spotify.com/api/token",
        data=auth_data,
        headers=auth_headers
    )
    
    access_token = auth_response.json().get("access_token")

    # Step 3: Use the access token to create a private playlist
    playlist_headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    playlist_data = {
        "name": "My Private Playlist",
        "public": False
    }
    
    playlist_response = requests.post(
        "https://api.spotify.com/v1/me/playlists",
        json=playlist_data,
        headers=playlist_headers
    )
    
    return "Private playlist created!"

if __name__ == "__main__":
    app.run(debug=True)


In [4]:
from flask import Flask, redirect
import random
import string
import urllib.parse

app = Flask(__name__)

# CLIENT_ID = SPOTIPY_CLIENT_ID
REDIRECT_URI = 'http://localhost:8889/lab'

def generate_random_string(length):
    letters = string.ascii_letters + string.digits
    return ''.join(random.choice(letters) for _ in range(length))

@app.route('/login')
def login():
    state = generate_random_string(16)
    scope = 'playlist-modify-public playlist-modify-private'

    redirect_url = (
        'https://accounts.spotify.com/authorize?' +
        urllib.parse.urlencode({
            'response_type': 'code',
            'client_id': CLIENT_ID,
            'scope': scope,
            'redirect_uri': REDIRECT_URI,
            'state': state
        })
    )
    return redirect(redirect_url)

if __name__ == '__main__':
    app.run(port=9999)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:9999
Press CTRL+C to quit
127.0.0.1 - - [16/Aug/2023 11:43:06] "GET /login HTTP/1.1" 302 -
127.0.0.1 - - [16/Aug/2023 11:48:42] "GET /login HTTP/1.1" 302 -
127.0.0.1 - - [16/Aug/2023 11:52:19] "GET /login HTTP/1.1" 302 -
127.0.0.1 - - [16/Aug/2023 11:53:56] "GET /login HTTP/1.1" 302 -


In [21]:
# URLS
AUTH_URL = 'https://accounts.spotify.com/authorize'
TOKEN_URL = 'https://accounts.spotify.com/api/token'
BASE_URL = 'https://api.spotify.com/v1/'


# Make a request to the /authorize endpoint to get an authorization code
auth_code = requests.get(AUTH_URL, {
    'client_id': CLIENT_ID,
    'response_type': 'code',
    'redirect_uri': 'https://open.spotify.com/collection/playlists',
    'scope': 'playlist-modify-private playlist-modify-public',
})
print(auth_code)

auth_header = base64.urlsafe_b64encode((CLIENT_ID + ':' + CLIENT_SECRET).encode())
print(auth_header.decode())
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Authorization': 'Bearer %s' % auth_header.decode()
}

payload = {
    'grant_type': 'authorization_code',
    'code': auth_code,
    'redirect_uri': 'https://open.spotify.com/collection/playlists',
#     'client_id': CLIENT_ID,
#     'client_secret': CLIENT_SECRET,
}

# Make a request to the /token endpoint to get an access token
access_token_request = requests.post(url=TOKEN_URL, data=payload, headers=headers)

# convert the response to JSON
access_token_response_data = access_token_request.json()

print(access_token_response_data)

# save the access token
access_token = access_token_response_data['access_token']

<Response [200]>
NmUxYTIyYTMxYzVkNDhjYTg0MjlkYmFjM2I0YTAzMzI6NzM3NGZjM2FmZGQ0NDY5YThjNzJmNzI0NGFiMjA5Yjg=
{'error': 'invalid_client'}


KeyError: 'access_token'

In [10]:
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
import requests
import uvicorn


# client_id = "YOUR_CLIENT_ID"
# client_secret = "YOUR_CLIENT_SECRET"
# redirect_uri = "YOUR_REDIRECT_URI" # e.g. http://localhost:8000/callback/ --> you will have to whitelist this url in the spotify developer dashboard 

app = FastAPI()

def get_access_token(auth_code: str):
    response = requests.post(
        "https://accounts.spotify.com/api/token",
        data={
            "grant_type": "authorization_code",
            "code": auth_code,
            "redirect_uri": redirect_uri,
        },
        auth=(client_id, client_secret),
    )
    access_token = response.json()["access_token"]
    return {"Authorization": "Bearer " + access_token}


@app.get("/")
async def auth():
    scope = ["playlist-modify-private", "playlist-modify-public"]
    auth_url = f"https://accounts.spotify.com/authorize?response_type=code&client_id={client_id}&redirect_uri={redirect_uri}&scope={' '.join(scope)}"
    return HTMLResponse(content=f'<a href="{auth_url}">Authorize</a>')


@app.get("/callback")
async def callback(code):

    headers = get_access_token(code)
    response = requests.get("https://api.spotify.com/v1/me", headers=headers)
    user_id = response.json()["id"]


    name = "Name of your playlist"
    description = "Description of your playlist"

    params = {
        "name": name,
        "description": description,
        "public": True,
    }

    url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    response = requests.post(url=url, headers=headers, json=params)
    playlist_id = response.json()["id"]

    track_uri = "spotify:track:319eU2WvplIHzjvogpnNc6"
    response = requests.post(
        f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks",
        headers=headers,
        json={"uris": [track_uri]},
    )
    if response.status_code == 201:
        return {"message": "Track added successfully!"}
    else:
        return {"error": response.json()}


if __name__ == "__main__":
    uvicorn.run(app, debug=True)

ImportError: cannot import name 'TypeAliasType' from 'typing_extensions' (/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/typing_extensions.py)